In [1]:
# Find project root
import subprocess, os
cmd = 'git rev-parse --show-toplevel'
root = subprocess.check_output(cmd, shell=True, text=True)
os.chdir(root.strip())
print('CWD:', os.getcwd())

CWD: /home/lab/cart_pole


In [2]:
from sessions.collector import SessionData
from misc.simple_server import session_widget
from pathlib import Path
import pandas
import struct
import json
import math

PATH = Path('data/sessions/calibration_test_1')
DATA = SessionData.load(PATH / 'session.json')

In [4]:
#DATA.values["state.position"].x

In [5]:
# abs_pole_angles = []
# rotations_arr = []
# rotations = 0
# prev = 0
# max_delta = math.pi
# for curr in DATA.values['state.pole_angle'].y:
#     delta = curr - prev
#     if delta > max_delta:
#         rotations -= 1
#     elif delta < -max_delta:
#         rotations += 1
#     abs_angle = 2 * math.pi * rotations + curr
#     abs_pole_angles.append(abs_angle)
#     rotations_arr.append(2 * math.pi * rotations)
#     prev = curr
    
# id = 'abs_pole_angle'
# DATA.values[id] = SessionData.Value(id, id, 'rad', DATA.values['state.pole_angle'].x, abs_pole_angles)
# id = 'pole_rotations'
# DATA.values[id] = SessionData.Value(id, id, 'rad', DATA.values['state.pole_angle'].x, rotations_arr)

In [6]:
DATA.groups = [
    SessionData.Group(name='Position', values=['state.cart_position']),
    SessionData.Group(name='Velocity', values=['state.cart_velocity']),
    SessionData.Group(name='Acceleration', values=['state.cart_acceleration', 'target.acceleration']),
#     SessionData.Group(name='Pole angle', values=['state.pole_angle']),
#     SessionData.Group(name='Pole velocity', values=['state.pole_velocity']),
]
session_widget(DATA)